In [1]:
import scapy.all as sc
import numpy as np
import pandas as pd
from scipy.stats import entropy

In [2]:
SNIFF_TIMEOUT = 1200
BROADCAST = "BROADCAST"
UNICAST = "UNICAST"
PCAP_FILE = "packages.pcap"
ARP_OP_WHO_HAS = 1
ARP_OP_IS_AT = 2

## Hago sniff durante sniff_timeout segundos

In [3]:
packages = sc.sniff(timeout=SNIFF_TIMEOUT)

## Guardo los paquetes en formato pcap

In [4]:
sc.wrpcap(PCAP_FILE,packages)

## Leo los paquetes del archivo .pcap

In [5]:
packages = sc.sniff(offline=PCAP_FILE)

## Ejercicio 3.2.1

In [6]:
def simbolo_s(package):
    if p.dst == 'ff:ff:ff:ff:ff:ff':
        return BROADCAST
    else:
        return UNICAST

In [7]:
def entropia_fuente(fuente):
    counts = fuente.value_counts()
    return entropy(base=2,pk=counts.as_matrix())

In [8]:
def informacion_simbolos(fuente):
    counts = fuente.value_counts()
    total = counts.sum()
    probs = counts / float(total)
    return pd.Series(- np.log2(probs), index=counts.index)

In [9]:
fuente = pd.Series([simbolo_s(p) for p in packages])
contadores = fuente.value_counts()
print "Entropia fuente S =", entropia_fuente(fuente)
print contadores
print informacion_simbolos(fuente)

Entropia fuente S = 0.615759989127
UNICAST      20832
BROADCAST     3745
dtype: int64
UNICAST      0.238507
BROADCAST    2.714271
dtype: float64


## Ejercicio 3.2.2

In [10]:
arp_packages = sc.sniff(offline=PCAP_FILE, lfilter=lambda x: x.haslayer(sc.ARP))

In [24]:
arp_packages = map(lambda p: p["ARP"], arp_packages)
who_has_packages = filter(lambda p : p.op == ARP_OP_WHO_HAS,arp_packages)
is_at_packages = filter(lambda p : p.op == ARP_OP_IS_AT, arp_packages)

In [32]:
is_at_packages[10]

<ARP  hwtype=0x1 ptype=0x800 hwlen=6 plen=4 op=is-at hwsrc=a0:ce:c8:03:1d:eb psrc=10.254.213.103 hwdst=00:90:9a:9d:ab:33 pdst=10.254.213.28 |<Padding  load='\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00' |>>

In [26]:
who_has_packages[0]

<ARP  hwtype=0x1 ptype=0x800 hwlen=6 plen=4 op=who-has hwsrc=c4:da:26:05:7c:75 psrc=10.254.213.39 hwdst=00:00:00:00:00:00 pdst=10.254.213.254 |<Padding  load='\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00' |>>

## Analisis de simbolos distinguidos utilizando who-has packages

In [27]:
def simbolo_s2(package):
    return package.pdst

In [35]:
fuente_s2 = pd.Series([simbolo_s2(p) for p in who_has_packages])
contadores = fuente_s2.value_counts()
print contadores
print "Entropia de la fuente =", entropia_fuente(fuente_s2)
print "Informacion simbolos"
print informacion_simbolos(fuente_s2).iloc[:]

10.254.213.254     137
10.254.95.254       57
10.254.213.74       43
169.254.255.255     29
10.254.213.28       14
10.254.213.41       10
10.254.213.63        8
10.254.213.49        5
10.254.213.88        4
10.254.213.45        4
10.254.213.66        2
10.254.213.97        2
10.254.213.103       2
10.254.213.25        1
10.254.213.57        1
dtype: int64
Entropia de la fuente = 2.60190211277
Informacion simbolos
10.254.213.254     1.219381
10.254.95.254      2.484523
10.254.213.74      2.891148
169.254.255.255    3.459432
10.254.213.28      4.510058
10.254.213.41      4.995485
10.254.213.63      5.317413
10.254.213.49      5.995485
10.254.213.88      6.317413
10.254.213.45      6.317413
10.254.213.66      7.317413
10.254.213.97      7.317413
10.254.213.103     7.317413
10.254.213.25      8.317413
10.254.213.57      8.317413
dtype: float64


## Analisis de simbolos distinguidos utilizando is_at packages

In [29]:
def simbolo_s3(package):
    return package.psrc

In [30]:
fuente_s3 = pd.Series([simbolo_s3(p) for p in is_at_packages])
contadores_s3 = fuente_s3.value_counts()
print contadores_s3
print "Entropia de la fuente =", entropia_fuente(fuente_s3)
print informacion_simbolos(fuente_s3)

10.254.213.74     43
10.254.213.103     1
dtype: int64
Entropia de la fuente = 0.156491062906
10.254.213.74     0.033167
10.254.213.103    5.459432
dtype: float64


In [31]:
len(is_at_packages)

44